## 1. 데이터 로드

In [33]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
import matplotlib.pyplot as plt

In [2]:
purpose = pd.read_csv("../data/수정데이터/용도별 목적대화 데이터.csv")
topic = pd.read_csv("../data/수정데이터/주제별 텍스트 일상 대화 데이터.csv")
sense = pd.read_csv("../data/수정데이터/일반상식 데이터.csv")
info = pd.read_csv("../data/수정데이터/여행정보 데이터.csv")
recommend = pd.read_csv("../data/수정데이터/여행지추천 데이터.csv")

purpose.dropna(inplace=True)
topic.dropna(inplace=True)
sense.dropna(inplace=True)
info.dropna(inplace=True)
recommend.dropna(inplace=True)

In [8]:
print(f'purpose shape: {purpose.shape}')
print(f'topic shape: {topic.shape}')
print(f'sense shape: {sense.shape}')
print(f'info shape: {info.shape}')
print(f'recommend shape: {recommend.shape}')

purpose shape: (225974, 1)
topic shape: (870138, 1)
sense shape: (68538, 3)
info shape: (11400, 3)
recommend shape: (980, 2)


In [9]:
print(purpose.columns)
print(topic.columns)
print(sense.columns)
print(info.columns)
print(recommend.columns)

Index(['text'], dtype='object')
Index(['text'], dtype='object')
Index(['intent', 'question', 'answer'], dtype='object')
Index(['city', 'title', 'label'], dtype='object')
Index(['question', 'answer'], dtype='object')


In [77]:
total_data = list(purpose['text']) + list(topic['text']) + list(sense['question']) + list(sense['answer']) + \
            list(info['city']) + list(info['title']) + list(recommend['question']) + list(recommend['answer'])

In [78]:
len(total_data)

1257948

In [79]:
df = pd.DataFrame({'text': total_data})
df.to_csv('../data/수정데이터/통합데이터.csv', index=False)

## 2. 의도 분류 데이터 생성
* 0: 추천
* 1: 예약
* 2: 정보

In [3]:
# import zipfile
# with zipfile.ZipFile('../data/원본데이터/여행 정보 데이터셋.zip') as zip_ref:
#     zip_ref.extractall('../data/원본데이터/')

In [1]:
import pandas as pd
info_df = pd.read_excel('../data/원본데이터/ko_template.xlsx')

In [2]:
info_df.head()

,type,city,title,start date,start time,end date,place
0,관광지,전라남도,‘봄의 왈츠’ 촬영지,NaN,NaN,NaN,전라남도 완도군 청산면 청산로 136
1,관광지,전라남도,‘서편제’ 촬영지,NaN,NaN,NaN,전라남도 완도군 청산면 당락리
2,관광지,강원도,㈜강원심층수,NaN,NaN,NaN,강원도 고성군 죽왕면 심층수길 124-19
3,관광지,강원도,㈜그래미 발명역사관,NaN,NaN,NaN,강원도 철원군 갈말읍 갈말로 32-30
4,관광지,제주도,㈜제주사랑 농수산,NaN,NaN,NaN,제주특별자치도 제주시 구좌읍 행원로13길 67


In [3]:
city = info_df['city'].unique().tolist()
city

['전라남도',
 '강원도',
 '제주도',
 '경상남도',
 '부산',
 '경기도',
 '충청북도',
 '전라북도',
 '경상북도',
 '충청남도',
 '대전',
 '울산',
 '대구',
 '서울',
 '인천',
 '광주',
 '세종특별자치시']

In [5]:
city2 = info_df['title'].str.split(" ").str.get(i=0).unique().tolist()
city2

['‘봄의',
 '‘서편제’',
 '㈜강원심층수',
 '㈜그래미',
 '㈜제주사랑',
 '㈜제주오렌지',
 '가거도(소흑산도)',
 '가계해변',
 '가고파',
 '가나안목장',
 '가덕도',
 '가덕산',
 '가라산',
 '가락공방',
 '가령폭포',
 '가루매마을',
 '가리산',
 '가리산(포천)',
 '가리산(홍천)',
 '가리산자연휴양림',
 '가리산자연휴양림캠핑장',
 '가리왕산',
 '가마골계곡',
 '가마골생태공원',
 '가마미해수욕장',
 '가마소계곡',
 '가마오름',
 '가메창(암메)',
 '가뫼골마을',
 '가문이오름(감은이오름)',
 '가배량성',
 '가사도',
 '가사동백숲해변',
 '가사해수욕장',
 '가산공원',
 '가산사',
 '가산서원',
 '가새기오름',
 '가선정',
 '가섭사',
 '가섭암지',
 '가세오름',
 '가시오가피마을',
 '가시오름',
 '가실성당',
 '가야동계곡',
 '가야산',
 '가야산(서산)',
 '가야산국립공원(백운동',
 '가야산국립공원(홍류동,청량동지구)',
 '가야산칠불봉',
 '가야의',
 '가야진사',
 '가양비래근린공원',
 '가오실공원',
 '가우도',
 '가원미술관',
 '가원습지',
 '가의도',
 '가인',
 '가정녹색농촌체험마을',
 '가조',
 '가지산',
 '가지산도립공원(밀양)',
 '가지산도립공원(울주)',
 '가진항',
 '가진해변',
 '가창',
 '가천',
 '가칠봉지구전투전적지',
 '가파도',
 '가평',
 '가평군목공예영농조합',
 '가평리',
 '가평별묘',
 '가평삼회',
 '가평의병3.1항일운동기념비',
 '가평학도의용대참전기념비',
 '가평향교',
 '가학동지석묘',
 '가학산',
 '가학산자연휴양림',
 '가호서원',
 '각사뽈똥체험마을',
 '각산',
 '각시바우오름',
 '각연사(괴산)',
 '각원사',
 '각화산',
 '각흘도',
 '간곡서원',
 '간성향교',
 '간월도',
 '간월도마을',
 '간월산',
 '간월암(서산)',
 '간재선생유지',


In [17]:
place = []
time = []
etc = []
nan = []

In [3]:
import pandas as pd
total_data = pd.read_csv('../data/수정데이터/통합데이터.csv')

In [19]:
place_keywords = ['국내', '여행', '장소', '주소', '위치', '추천', '어디', '근처', '맛집','제주','세종','충북','충남','강원','경남','경북','전남','전북']
time_keywords = ['봄', '여름', '가을', '겨울', '시간', '기간', '언제', '계획','1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월',
                 '월요일','화요일','수요일','목요일','금요일','토요일','일요일','이번 주','다음 주','다음 날','내일','다음 달']
place_keywords+=city
place_keywords+=city2

In [4]:
total_data.head(50)

,text
0,반갑습니다 소속 상담사 이름 입니다
1,안내해주신 방법을 시도해 봤는데 안 돼서요
2,그러시다면 두 번째 방법은요
3,휴대폰 명의자 본인이요
4,휴대폰하고 신분증 챙기셔서 저희 서비스센터 쪽으로 방문 하셔야 되세요 고객님
5,이게 왜 이렇게 된거죠
6,그거 내가 설정 안 해놨는데요
7,그건 고객님 설정하신 번호 눌러도 진입이 안 되세요 고객님
8,그거 해봤지요 안 되네요
9,일단 고객님 고객님께서 설정하신 걸 입력하셨는 데도 안 되신다면요


In [21]:
for data in total_data['text']:
    try:
        if any(keyword in data for keyword in place_keywords):
            place.append(data)
        elif any(keyword in data for keyword in time_keywords):
            time.append(data)
        else:
            etc.append(data)
    except:
        nan.append(data)

In [22]:
print(len(place),len(time),len(etc))

314163 44998 898787


In [23]:
place_df = pd.DataFrame({'text': place})
place_df['label'] = 0
place_df.head()

,text,label
0,예 고객님 회현지구 쪽으로 확인해 드리겠습니다,0
1,저희가 부산 해운대구에는 주소 에 있는 센타가 있고요,0
2,아니시면 주소 에 있는 작은 분점도 확인되고 있습니다,0
3,주소 이 나을 거 같아요,0
4,그러세요 그러시다면 주소 은 지하철 장산역 십 사 번 출구에 있구요,0


In [24]:
time_df = pd.DataFrame({'text': time})
time_df['label'] = 1
time_df.tail()

,text,label
44993,2005년 11월 16일,1
44994,1995년 12월 25일,1
44995,일요일과 공휴일,1
44996,2010년 7월부로 징병제를 폐지,1
44997,1905년 2월 1일,1


In [25]:
etc_df = pd.DataFrame({'text': etc})
etc_df['label'] = 2
etc_df

,text,label
0,반갑습니다 소속 상담사 이름 입니다,2
1,안내해주신 방법을 시도해 봤는데 안 돼서요,2
2,그러시다면 두 번째 방법은요,2
3,휴대폰 명의자 본인이요,2
4,휴대폰하고 신분증 챙기셔서 저희 서비스센터 쪽으로 방문 하셔야 되세요 고객님,2
...,...,...
898782,당일인가요 박 일 인가요 채택 부탁드립니다 알아두세요 위 답변은 답변작성자가 경...,2
898783,답변이 질문자에 의해 채택된 이후에는 답변 내용의 수정 및 삭제가 제한될 수 있사오...,2
898784,남한 산성도 괜찮던데요 알아두세요 위 답변은 답변작성자가 경험과 지식을 바탕으...,2
898785,당일인가요 박 일 인가요 채택 부탁드립니다 알아두세요 위 답변은 답변작성자가 경...,2


In [26]:
total_train_data = pd.concat([place_df, time_df, etc_df], axis=0)
total_train_data.reset_index(drop=True, inplace=True)
total_train_data.shape

(1257948, 2)

In [27]:
total_train_data[total_train_data['label'] == 0]

,text,label
0,예 고객님 회현지구 쪽으로 확인해 드리겠습니다,0
1,저희가 부산 해운대구에는 주소 에 있는 센타가 있고요,0
2,아니시면 주소 에 있는 작은 분점도 확인되고 있습니다,0
3,주소 이 나을 거 같아요,0
4,그러세요 그러시다면 주소 은 지하철 장산역 십 사 번 출구에 있구요,0
...,...,...
314158,안녕하세요 부산에 가볼만한곳도많고 맛집도 너무많아서 고민이 많이되실꺼에요 ㅜㅜ 부...,0
314159,저는 강원도에 안번데기 다녀왔는데 너무 좋았어서 추천드립니다 답변해드리면서 강원도...,0
314160,포항에 일본가옥거리 추천해드릴게요 작은 골목이라서 넓지는 않지만 옛날 생각나는 불...,0
314161,국제슬로시티로 지정된 전주 한옥마을 추천합니다 용산역에서 전라선기차와 시외 고속버...,0


In [28]:
total_train_data[total_train_data['label'] == 1]

,text,label
314163,언제 쯤 연락을 해주셨나요 담당자님이랑요,1
314164,네 죄송하지만 이번 주까지는 저희도 확인할 방법이 없네요,1
314165,알겠습니다 그러면 이번 주까지는 기다려볼게요,1
314166,내일까지 입금하면 돼요,1
314167,알았습니다 그러면 이거는 내일 안 도착 가능한가요,1
...,...,...
359156,2005년 11월 16일,1
359157,1995년 12월 25일,1
359158,일요일과 공휴일,1
359159,2010년 7월부로 징병제를 폐지,1


In [29]:
total_train_data[total_train_data['label'] == 2]

,text,label
359161,반갑습니다 소속 상담사 이름 입니다,2
359162,안내해주신 방법을 시도해 봤는데 안 돼서요,2
359163,그러시다면 두 번째 방법은요,2
359164,휴대폰 명의자 본인이요,2
359165,휴대폰하고 신분증 챙기셔서 저희 서비스센터 쪽으로 방문 하셔야 되세요 고객님,2
...,...,...
1257943,당일인가요 박 일 인가요 채택 부탁드립니다 알아두세요 위 답변은 답변작성자가 경...,2
1257944,답변이 질문자에 의해 채택된 이후에는 답변 내용의 수정 및 삭제가 제한될 수 있사오...,2
1257945,남한 산성도 괜찮던데요 알아두세요 위 답변은 답변작성자가 경험과 지식을 바탕으...,2
1257946,당일인가요 박 일 인가요 채택 부탁드립니다 알아두세요 위 답변은 답변작성자가 경...,2


In [30]:
total_train_data.to_csv('../data/수정데이터/total_train_data.csv', index=False)

## 3. 적절한 패딩 길이 탐색

In [31]:
total_train_data = pd.read_csv('../data/수정데이터/total_train_data.csv')

In [34]:
tokenizer = Komoran()

In [35]:
data_tokenized = [[token + '/' + POS for token, POS in tokenizer.pos(text)] for text in total_train_data['text']]

In [36]:
exclusion_tags = [
    "JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ",
    "JX", "JC",
    "SF", "SP", "SS", "SE", "SO",
    "EP", "EF", "EC", "ETN", "ETM",
    "XSN", "XSV", "XSA",
]
def Exclusion_tags(x):
    return x in exclusion_tags

data_list = []
for i in range(len(data_tokenized)):
    temp = []
    for j in range(len(data_tokenized[i])):
        if Exclusion_tags(data_tokenized[i][j].split('/')[1]) is False:
            temp.append(data_tokenized[i][j].split('/')[0])
    data_list.append(temp)

In [37]:
len(data_list)

1257948

In [38]:
num_tokens = [len(sent) for sent in data_list]
num_tokens = np.array(num_tokens)
print('토큰 길이 평균:', num_tokens.mean())
print('토큰 길이 최대:', num_tokens.max())
print('토큰 길이 표준편차:', num_tokens.std())

토큰 길이 평균: 6.450725308200339
토큰 길이 최대: 1984
토큰 길이 표준편차: 9.888866319442956


In [39]:
MAX_SEQ_LEN = 15
def below_len_rate(max_len, data_list):
    cnt = 0
    for s in data_list:
        if (len(s) <= max_len):
            cnt = cnt + 1

    print(f'전체 샘플 중 길이가 {max_len} 이하인 샘플의 비율: {(cnt / len(data_list))}')
below_len_rate(MAX_SEQ_LEN, data_list)

전체 샘플 중 길이가 15 이하인 샘플의 비율: 0.9868945298215824
